In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Custom Orthogonal Regularizer
C = 1e-4

def orthogonal_reg(w):
    units = w.shape[-1]
    w = w.view(-1, units)
    eye = torch.eye(units, device=w.device)
    return (C / 2) * torch.norm(w.T @ w - eye)

# Dropout tail function
def dropout_tail(X):
    total_dim = X.shape[-1]
    tail_len = torch.randint(0, total_dim + 1, (1,))
    head_len = total_dim - tail_len
    mask = torch.cat((torch.ones(X.shape[1], X.shape[2], head_len.item()), 
                      torch.zeros(X.shape[1], X.shape[2], tail_len.item())), dim=-1).to(X.device)
    return X * mask

# Encoder Model
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=9, stride=7, padding=4)
        self.conv2 = nn.Conv2d(16, 10, kernel_size=3, stride=1, padding=1)
        nn.init.orthogonal_(self.conv1.weight)
        nn.init.orthogonal_(self.conv2.weight)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# Decoder Model
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.conv_transpose1 = nn.ConvTranspose2d(10, 64, kernel_size=9, stride=7, padding=4, output_padding=6 )
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2)
        self.conv4 = nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = F.relu(self.conv_transpose1(x))
        x = F.relu(self.conv2(x)) + x
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv3(x)) + x
        x = self.conv4(x)
        x = torch.clamp(x, min=0, max=1)
        return x


In [2]:

# Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [3]:
# Example usage
autoencoder = Autoencoder()
print(autoencoder)


Autoencoder(
  (encoder): Encoder(
    (conv1): Conv2d(3, 16, kernel_size=(9, 9), stride=(7, 7), padding=(4, 4))
    (conv2): Conv2d(16, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (decoder): Decoder(
    (conv_transpose1): ConvTranspose2d(10, 64, kernel_size=(9, 9), stride=(7, 7), padding=(4, 4))
    (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (conv4): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

# Define paths for the encoder and decoder models
encoder_model_dir = "joint_ae/best_model_save_encoder"
decoder_model_dir = "joint_ae/best_model_save_decoder"

# Load the encoder model and inspect variables
tf_encoder_model = tf.saved_model.load(encoder_model_dir)
print("Encoder model loaded from:", encoder_model_dir)
print("Encoder model variables:")
for var in tf_encoder_model.variables:
    print(f"  {var.name}: {var.shape}")

# Load the decoder model and inspect variables
tf_decoder_model = tf.saved_model.load(decoder_model_dir)
print("\nDecoder model loaded from:", decoder_model_dir)
print("Decoder model variables:")
for var in tf_decoder_model.variables:
    print(f"  {var.name}: {var.shape}")



Encoder model loaded from: joint_ae/best_model_save_encoder
Encoder model variables:
  conv2d/kernel:0: (9, 9, 3, 16)
  conv2d/bias:0: (16,)
  encoder_out/kernel:0: (3, 3, 16, 10)
  encoder_out/bias:0: (10,)

Decoder model loaded from: joint_ae/best_model_save_decoder
Decoder model variables:
  decoder_input/kernel:0: (9, 9, 64, 10)
  decoder_input/bias:0: (64,)
  conv2d_1/kernel:0: (5, 5, 64, 64)
  conv2d_1/bias:0: (64,)
  conv2d_2/kernel:0: (5, 5, 64, 64)
  conv2d_2/bias:0: (64,)
  conv2d_3/kernel:0: (5, 5, 64, 64)
  conv2d_3/bias:0: (64,)
  conv2d_4/kernel:0: (3, 3, 64, 3)
  conv2d_4/bias:0: (3,)


In [5]:
# Dictionary to store TensorFlow variables by name for easier lookup for `loaded`
tf_weights_encoder = {v.name: v.numpy() for v in tf_encoder_model.variables}

# Load weights for the Encoder from `tf_weights_encoder`
with torch.no_grad():
    # Load weights and biases for conv1
    autoencoder.encoder.conv1.weight.copy_(torch.tensor(tf_weights_encoder['conv2d/kernel:0']).permute(3, 2, 0, 1))
    autoencoder.encoder.conv1.bias.copy_(torch.tensor(tf_weights_encoder['conv2d/bias:0']))

    # Load weights and biases for conv2
    autoencoder.encoder.conv2.weight.copy_(torch.tensor(tf_weights_encoder['encoder_out/kernel:0']).permute(3, 2, 0, 1))
    autoencoder.encoder.conv2.bias.copy_(torch.tensor(tf_weights_encoder['encoder_out/bias:0']))

# Dictionary to store `loaded2` TensorFlow variables by name for the decoder part
tf_weights_decoder = {v.name: v.numpy() for v in tf_decoder_model.variables}

# Load weights for the Decoder from `tf_weights_decoder`
with torch.no_grad():
    # Load weights and biases for conv_transpose1
    autoencoder.decoder.conv_transpose1.weight.copy_(torch.tensor(tf_weights_decoder['decoder_input/kernel:0']).permute(3, 2, 0, 1))
    autoencoder.decoder.conv_transpose1.bias.copy_(torch.tensor(tf_weights_decoder['decoder_input/bias:0']))

    # Load weights and biases for conv2
    autoencoder.decoder.conv2.weight.copy_(torch.tensor(tf_weights_decoder['conv2d_1/kernel:0']).permute(3, 2, 0, 1))
    autoencoder.decoder.conv2.bias.copy_(torch.tensor(tf_weights_decoder['conv2d_1/bias:0']))

    # Load weights and biases for conv3
    autoencoder.decoder.conv3.weight.copy_(torch.tensor(tf_weights_decoder['conv2d_2/kernel:0']).permute(3, 2, 0, 1))
    autoencoder.decoder.conv3.bias.copy_(torch.tensor(tf_weights_decoder['conv2d_2/bias:0']))

    # Load weights and biases for conv4
    autoencoder.decoder.conv4.weight.copy_(torch.tensor(tf_weights_decoder['conv2d_4/kernel:0']).permute(3, 2, 0, 1))
    autoencoder.decoder.conv4.bias.copy_(torch.tensor(tf_weights_decoder['conv2d_4/bias:0']))

print("Weights loaded successfully into autoencoder.encoder and autoencoder.decoder.")

Weights loaded successfully into autoencoder.encoder and autoencoder.decoder.


In [6]:
torch.save(autoencoder.state_dict(), 'pnc_autoencoder_weights.pth')
torch.save(autoencoder.encoder.state_dict(), 'pnc_encoder_weights.pth')
torch.save(autoencoder.decoder.state_dict(), 'pnc_decoder_weights.pth')